# Engenharia de Atributos

Importando as dependências

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Lendo os dados brutos e concatenando-os em um único dataframe

In [148]:
base_path = '../data/raw/' 

dados_2021 = pd.read_csv(base_path + '2021.csv', encoding='latin-1', sep=';')
dados_2022 = pd.read_csv(base_path + '2022.csv', sep=',')
dados_2023 = pd.read_csv(base_path + '2023.csv', encoding='latin-1', sep=';')
dados_2024 = pd.read_csv(base_path + '2024.csv', sep=',')

dados = pd.concat([dados_2021, dados_2022, dados_2023, dados_2024], axis=0, ignore_index=True)

dados.shape

(250889, 30)

Na análise exploratória, foram indentificadas colunas em que haviam dados nulos:

In [149]:
nulos = dados.isnull().sum().sort_values(ascending=False)
nulos[nulos>0]

uop                       219
delegacia                  85
regional                   23
classificacao_acidente      4
dtype: int64

Reconhecendo os dados dessas colunas

In [150]:
dados['uop'].head(5), dados['delegacia'].head(5), dados['regional'].head(5), dados['classificacao_acidente'].head(5)

(0    UOP01-DEL05-SP
 1    UOP01-DEL01-AM
 2    UOP03-DEL04-CE
 3    UOP02-DEL04-RJ
 4    UOP04-DEL05-RJ
 Name: uop, dtype: object,
 0    DEL05-SP
 1    DEL01-AM
 2    DEL04-CE
 3    DEL04-RJ
 4    DEL05-RJ
 Name: delegacia, dtype: object,
 0    SPRF-SP
 1    SPRF-AM
 2    SPRF-CE
 3    SPRF-RJ
 4    SPRF-RJ
 Name: regional, dtype: object,
 0    Com Vítimas Feridas
 1                    NaN
 2    Com Vítimas Feridas
 3    Com Vítimas Feridas
 4    Com Vítimas Feridas
 Name: classificacao_acidente, dtype: object)

Substituindo os dados faltantes pela moda das respectivas colunas, mas considerando a unidade federativa específica para a substituição assim os dados ficarão mais precisos. Ao mesmo tempo, para aproveitar o laço de repetição, vamos criar a coluna periodo que constará a informação do período do dia que o acidente ocorreu.

In [151]:
zero_h = pd.to_datetime('00:00:00', format='%H:%M:%S').time()
seis_h = pd.to_datetime('06:00:00', format='%H:%M:%S').time()
doze_h = pd.to_datetime('12:00:00', format='%H:%M:%S').time()
dezoito_h = pd.to_datetime('18:00:00', format='%H:%M:%S').time()

for index, row in dados.iterrows():

    if pd.isnull(row['uop']):

        uf_especifica = row['uf'] 
        aux_df = dados[dados['uf'] == uf_especifica]
        if not aux_df['uop'].mode().empty:
            dados.at[index, 'uop'] = aux_df['uop'].mode()[0]

    if pd.isnull(row['delegacia']):

        uf_especifica = row['uf'] 
        aux_df = dados[dados['uf'] == uf_especifica]

        if not aux_df['delegacia'].mode().empty:
            dados.at[index, 'delegacia'] = aux_df['delegacia'].mode()[0]

    if pd.isnull(row['regional']):

        uf_especifica = row['uf'] 
        aux_df = dados[dados['uf'] == uf_especifica]

        if not aux_df['regional'].mode().empty:
            dados.at[index, 'regional'] = aux_df['regional'].mode()[0]

    if pd.isnull(row['classificacao_acidente']):

        uf_especifica = row['uf'] 
        aux_df = dados[dados['uf'] == uf_especifica]

        if not aux_df['classificacao_acidente'].mode().empty:
            dados.at[index, 'classificacao_acidente'] = aux_df['classificacao_acidente'].mode()[0]

    horario =  pd.to_datetime(row['horario'], format='%H:%M:%S').time()

    if horario >= zero_h and horario<seis_h:
        dados.at[index, 'periodo'] = 'madrugada'
    elif horario >= seis_h and horario < doze_h:
        dados.at[index, 'periodo'] = 'manhã'
    elif horario >= doze_h and horario < dezoito_h:
        dados.at[index, 'periodo'] = 'tarde'
    else:
        dados.at[index, 'periodo'] = 'noite'

Checando se os valores nulos foram realmente preenchidos. Como nenhuma coluna com valor nulo é impressa, não existem mais valores nulos na base.

In [152]:
nulos = dados.isnull().sum().sort_values(ascending=False)
nulos[nulos>0]


Series([], dtype: int64)

Conferindo o tipo de dados para cada coluna

In [153]:
tipos_dados = dados.dtypes

tipos_dados

id                        float64
data_inversa               object
dia_semana                 object
horario                    object
uf                         object
br                          int64
km                         object
municipio                  object
causa_acidente             object
tipo_acidente              object
classificacao_acidente     object
fase_dia                   object
sentido_via                object
condicao_metereologica     object
tipo_pista                 object
tracado_via                object
uso_solo                   object
pessoas                     int64
mortos                      int64
feridos_leves               int64
feridos_graves              int64
ilesos                      int64
ignorados                   int64
feridos                     int64
veiculos                    int64
latitude                   object
longitude                  object
regional                   object
delegacia                  object
uop           

Realizando conversões no tipo do dado para as colunas

In [154]:
# usando ponto (.) para delimitar casa decimal, padrão da linguagem python
dados['km'] = dados['km'].str.replace(',', '.').astype(float)
dados['uop'] = dados['uop'].astype('category')
dados['delegacia'] = dados['delegacia'].astype('category')
dados['regional'] = dados['regional'].astype('category')
dados['classificacao_acidente'] = dados['classificacao_acidente'].astype('category')
dados['periodo'] = dados['periodo'].astype('category')

print('Os novos tipos para os dados das colunas:')
dados[['km', 'uop', 'delegacia', 'regional', 'classificacao_acidente', 'periodo']].dtypes

Os novos tipos para os dados das colunas:


km                         float64
uop                       category
delegacia                 category
regional                  category
classificacao_acidente    category
periodo                   category
dtype: object

Tratamento de data/hora, mudando para o formato datetime padrão americano MM-DD-YYYY HH:MM AM/PM que geralmente são usados nos bancos de dados

In [155]:
dados['datetime'] = pd.to_datetime(dados['data_inversa'] + ' ' + dados['horario'])

# padrão americano, geralmente usado nos banco de dados
dados['datetime'] = dados['datetime'].dt.strftime('%m-%d-%Y %I:%M %p')

dados = dados.drop(columns=['data_inversa', 'horario'])

Verificando a base de dados antes de persistí-la

In [156]:
dados.shape

(250889, 30)

In [157]:
dados

,id,dia_semana,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,...,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,periodo,datetime
0,331730.0,sexta-feira,SP,116,453.0,REGISTRO,Reação tardia ou ineficiente do condutor,Tombamento,Com Vítimas Feridas,Amanhecer,...,0,1,1,"-24,56168114","-47,86752174",SPRF-SP,DEL05-SP,UOP01-DEL05-SP,madrugada,01-01-2021 05:30
1,331804.0,sexta-feira,AM,174,937.0,MANAUS,Reação tardia ou ineficiente do condutor,Colisão traseira,Com Vítimas Feridas,Pleno dia,...,1,1,5,"-2,508068","-60,036434",SPRF-AM,DEL01-AM,UOP01-DEL01-AM,manhã,01-01-2021 08:05
2,331815.0,sexta-feira,CE,222,128.5,ITAPAJE,Velocidade Incompatível,Tombamento,Com Vítimas Feridas,Pleno dia,...,11,2,12,"-3,707626","-39,623509",SPRF-CE,DEL04-CE,UOP03-DEL04-CE,manhã,01-01-2021 10:10
3,331823.0,sexta-feira,RJ,493,18.0,MAGE,Ingestão de álcool e/ou substâncias psicoativa...,Colisão lateral,Com Vítimas Feridas,Pleno dia,...,1,1,3,"-22,6687122","-43,01862696",SPRF-RJ,DEL04-RJ,UOP02-DEL04-RJ,tarde,01-01-2021 12:30
4,331843.0,sexta-feira,RJ,393,252.0,BARRA DO PIRAI,Condutor Dormindo,Colisão frontal,Com Vítimas Feridas,Pleno dia,...,1,2,3,"-22,43422023","-43,77561487",SPRF-RJ,DEL05-RJ,UOP04-DEL05-RJ,tarde,01-01-2021 02:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250884,635554.0,sexta-feira,BA,324,615.0,SALVADOR,Condutor deixou de manter distância do veículo...,Colisão lateral mesmo sentido,Com Vítimas Feridas,Pleno dia,...,0,1,2,"-12,87206398","-38,42746403",SPRF-BA,DEL01-BA,UOP01-DEL01-BA,tarde,09-27-2024 04:15
250885,635630.0,domingo,RJ,116,108.0,GUAPIMIRIM,Condutor deixou de manter distância do veículo...,Colisão traseira,Com Vítimas Feridas,Plena Noite,...,0,1,2,"-22,535843","-42,99125",SPRF-RJ,DEL04-RJ,UOP01-DEL04-RJ,noite,09-15-2024 08:30
250886,635699.0,terça-feira,AP,210,3.0,MACAPA,Transitar na calçada,Colisão lateral sentido oposto,Sem Vítimas,Pleno dia,...,0,0,2,"0,08723365","-51,08569007",SPRF-AP,DEL01-AP,UOP01-DEL01-AP,tarde,08-13-2024 04:22
250887,635887.0,sábado,PR,277,679.0,MEDIANEIRA,Manobra de mudança de faixa,Capotamento,Com Vítimas Feridas,Plena Noite,...,0,1,2,"-25,31758044","-54,15684223",SPRF-PR,DEL05-PR,UOP01-DEL05-PR,noite,09-21-2024 06:50


Persistindo a base de dados processada em ../data/processed

In [158]:
dados.to_csv('../data/processed/dados.csv', index=False)
